# Import Libraries

In [ ]:
import random
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import norm
import sklearn
import xgboost
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier

# Load the Training and Testing Datasets

In [ ]:
train=pd.read_csv("C:\\Users\\DELL\\Desktop\\train.csv")
test=pd.read_csv("C:\\Users\\DELL\\Desktop\\test.csv")

# Preprocess the Dataset

In [ ]:
#Preprocessing the data.
le=LabelEncoder()
scaler=StandardScaler()
train['Severity']=le.fit_transform(train['Severity'])
train.iloc[:,0:10]=scaler.fit_transform(train.iloc[:,0:10])
test.iloc[:,0:10]=scaler.fit_transform(test.iloc[:,0:10])
#Dividing the data into train and test sets.
x_train=train.iloc[:,0:10]
y_train=train.iloc[:,-1]
x_test=test.iloc[:,0:10]
#initialising the return list of SVM parameters from PSO.
pos=list()
pos.append(0)
pos.append(0)
# function we are attempting to optimize (minimize)

# Code for Particle Swarm Optimisation

In [ ]:
def svmM(x0):
    svc=svm.SVC(C=x0[0],gamma=x0[1])
    x_tr,x_ts,y_tr, y_ts = train_test_split(x_train, y_train, test_size=0.2, random_state=0)
    svc.fit(x_tr,y_tr)
    y_pred=svc.predict(x_ts)
    return accuracy_score(y_ts,y_pred)*-1
def func1(x0):
    return svmM(x0)    
class Particle:
    def __init__(self,x0):
        self.position_i=[]          # particle position
        self.velocity_i=[]          # particle velocity
        self.pos_best_i=[]          # best position individual
        self.err_best_i=-1          # best error individual
        self.err_i=-1               # error individual

        for i in range(0,num_dimensions):
            self.velocity_i.append(random.uniform(-1,1))
            self.position_i.append(x0[i])

    # evaluate current fitness
    def evaluate(self,costFunc):
        self.err_i=costFunc(self.position_i)

        # check to see if the current position is an individual best
        if self.err_i < self.err_best_i or self.err_best_i==-1:
            self.pos_best_i=self.position_i
            self.err_best_i=self.err_i

    # update new particle velocity
    def update_velocity(self,pos_best_g):
        w=0.5       # constant inertia weight (how much to weigh the previous velocity)
        c1=1        # cognative constant
        c2=2        # social constant

        for i in range(0,num_dimensions):
            r1=random.random()
            r2=random.random()

            vel_cognitive=c1*r1*(self.pos_best_i[i]-self.position_i[i])
            vel_social=c2*r2*(pos_best_g[i]-self.position_i[i])
            self.velocity_i[i]=w*self.velocity_i[i]+vel_cognitive+vel_social

    # update the particle position based off new velocity updates
    def update_position(self,bounds):
        for i in range(0,num_dimensions):
            self.position_i[i]=self.position_i[i]+self.velocity_i[i]

            # adjust maximum position if necessary
            if self.position_i[i]>bounds[i][1]:
                self.position_i[i]=bounds[i][1]

            # adjust minimum position if neseccary
            if self.position_i[i] < bounds[i][0]:
                self.position_i[i]=bounds[i][0]                
class PSO():
    def __init__(self,costFunc,x0,bounds,num_particles,maxiter):
        global num_dimensions

        num_dimensions=len(x0)
        err_best_g=-1                   # best error for group
        pos_best_g=[]                   # best position for group

        # establish the swarm
        swarm=[]
        for i in range(0,num_particles):
            swarm.append(Particle(x0))

        # begin optimization loop
        i=0
        while i < maxiter:
            #print i,err_best_g
            # cycle through particles in swarm and evaluate fitness
            for j in range(0,num_particles):
                swarm[j].evaluate(costFunc)

                # determine if current particle is the best (globally)
                if swarm[j].err_i < err_best_g or err_best_g == -1:
                    pos_best_g=list(swarm[j].position_i)
                    err_best_g=float(swarm[j].err_i)

            # cycle through swarm and update velocities and position
            for j in range(0,num_particles):
                swarm[j].update_velocity(pos_best_g)
                swarm[j].update_position(bounds)
            i+=1

        # print final results
        print ('FINAL:')
        print (pos_best_g)
        print (err_best_g)
        pos[0]=pos_best_g[0]
        pos[1]=pos_best_g[1]
        

# Applying Base Machine Learning Models

In [ ]:
def MLPerceptron(x_train,y_train,x_test):
    mlp_clf=MLPClassifier(max_iter=2000)
    mlp_clf.fit(x_train,y_train)
    train_pred=mlp_clf.predict(x_train)
    test_pred=mlp_clf.predict(x_test)
    return test_pred,train_pred

def RandomForest(x_train,y_train,x_test):
    rf=RandomForestClassifier()
    rf.fit(x_train,y_train)
    train_pred=rf.predict(x_train)
    test_pred=rf.predict(x_test)
    return test_pred,train_pred

#Applying SVM + Particle Swarm Optimisation
def SVMachine(x_train,y_train,x_test):
    initial=[1.0,0.1]               # initial starting location [x1,x2...]
    bounds=[(1e-2,1e2),(1e-2,1e2)]  # input bounds [(x1_min,x1_max),(x2_min,x2_max)...]
    PSO(func1,initial,bounds,num_particles=15,maxiter=30)
    C=pos[0]
    gamma=pos[1]
    svc=svm.SVC(C=C,gamma=gamma)
    svc.fit(x_train,y_train)
    train_pred=svc.predict(x_train)
    test_pred=svc.predict(x_test)
    return test_pred,train_pred

# Predict the Base Classifier Results

In [ ]:
#Applying the 3 best base classification learners.
test_pred1,train_pred1=MLPerceptron(x_train,y_train,x_test)
test_pred2,train_pred2=SVMachine(x_train,y_train,x_test)
test_pred3,train_pred3=RandomForest(x_train,y_train,x_test)

In [ ]:
#converting results to dataframes.
train_pred1=pd.DataFrame(train_pred1)
test_pred1=pd.DataFrame(test_pred1)
train_pred2=pd.DataFrame(train_pred2)
test_pred2=pd.DataFrame(test_pred2)
train_pred3=pd.DataFrame(train_pred3)
test_pred3=pd.DataFrame(test_pred3)
#Applying the ensemble learner on the results of the base learners.(Also called META LEARNER).

# Apply Logisitic Regression Ensemble Classifier 

In [ ]:
model=LogisticRegression()
df_train=pd.concat([train_pred1,train_pred2,train_pred3],axis=1)
df_test=pd.concat([test_pred1,test_pred2,test_pred3],axis=1)
model.fit(df_train,y_train)

# Predict and store the Final Results

In [ ]:
#Final results prediction and writing. 
df=model.predict(df_test)
df=le.inverse_transform(df)
df=pd.DataFrame(df)
path="C:\\Users\\DELL\\Desktop\\final_ans.csv"
df.to_csv(path)